In [14]:
import pandas as pd
from pathlib import Path
import re
import nltk
import numpy as np
from collections import defaultdict
import ast


## Change this part based on your directory

In [23]:
_dir = Path("/restricted/projectnb/batmanlab/shawn24/PhD/Mammo-CLIP/src/codebase/data_csv")
csv = _dir/"vindr_detection_v1_folds.csv"

df = pd.read_csv(csv)
print(csv)
print(df.columns)
df.head(5)

/restricted/projectnb/batmanlab/shawn24/PhD/Mammo-CLIP/src/codebase/data_csv/vindr_detection_v1_folds.csv
Index(['patient_id', 'series_id', 'image_id', 'laterality', 'view', 'height',
       'width', 'breast_birads', 'breast_density', 'finding_categories',
       'finding_birads', 'xmin', 'ymin', 'xmax', 'ymax', 'split',
       'resized_xmin', 'resized_ymin', 'resized_xmax', 'resized_ymax', 'fold',
       'Architectural_Distortion', 'Asymmetry', 'Focal_Asymmetry',
       'Global_Asymmetry', 'Mass', 'Nipple_Retraction', 'No_Finding',
       'Skin_Retraction', 'Skin_Thickening', 'Suspicious_Calcification',
       'Suspicious_Lymph_Node', 'density'],
      dtype='object')


/var/tmp/ipykernel_54343/2166500492.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)


,patient_id,series_id,image_id,laterality,view,height,width,breast_birads,breast_density,finding_categories,...,Focal_Asymmetry,Global_Asymmetry,Mass,Nipple_Retraction,No_Finding,Skin_Retraction,Skin_Thickening,Suspicious_Calcification,Suspicious_Lymph_Node,density
0,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,4e3a578fe535ea4f5258d3f7f4419db8.png,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Mass'],...,0,0,1,0,0,0,0,0,0,2
1,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,dac39351b0f3a8c670b7f8dc88029364.png,R,MLO,3518,2800,BI-RADS 4,DENSITY C,['Mass'],...,0,0,1,0,0,0,0,0,0,2
2,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,c83f780904f25eacb44e9030f32c66e1.png,R,CC,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],...,0,1,0,0,0,0,0,0,0,2
3,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,893528bc38a0362928a89364f1b692fd.png,R,MLO,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],...,0,1,0,0,0,0,0,0,0,2
4,c3487424fee1bdd4515b72dc3fd69813,77619c914263eae44e9099f1ce07192c,318264c881bf12f2c1efe5f93920cc37.png,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Architectural Distortion'],...,0,0,0,0,0,0,0,0,0,2


In [24]:
df['finding_categories'] = df['finding_categories'].apply(ast.literal_eval)
df

,patient_id,series_id,image_id,laterality,view,height,width,breast_birads,breast_density,finding_categories,...,Focal_Asymmetry,Global_Asymmetry,Mass,Nipple_Retraction,No_Finding,Skin_Retraction,Skin_Thickening,Suspicious_Calcification,Suspicious_Lymph_Node,density
0,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,4e3a578fe535ea4f5258d3f7f4419db8.png,R,CC,3518,2800,BI-RADS 4,DENSITY C,[Mass],...,0,0,1,0,0,0,0,0,0,2
1,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,dac39351b0f3a8c670b7f8dc88029364.png,R,MLO,3518,2800,BI-RADS 4,DENSITY C,[Mass],...,0,0,1,0,0,0,0,0,0,2
2,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,c83f780904f25eacb44e9030f32c66e1.png,R,CC,3518,2800,BI-RADS 3,DENSITY C,[Global Asymmetry],...,0,1,0,0,0,0,0,0,0,2
3,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,893528bc38a0362928a89364f1b692fd.png,R,MLO,3518,2800,BI-RADS 3,DENSITY C,[Global Asymmetry],...,0,1,0,0,0,0,0,0,0,2
4,c3487424fee1bdd4515b72dc3fd69813,77619c914263eae44e9099f1ce07192c,318264c881bf12f2c1efe5f93920cc37.png,R,CC,3518,2800,BI-RADS 4,DENSITY C,[Architectural Distortion],...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20481,f2093a752e6b44df5990f5fd38c99dd2,2b1b2b8f48abab9819c0b3d091e152ee,ea732154d149f619b20070b78060ae65.png,R,CC,2812,2012,BI-RADS 2,DENSITY C,[No Finding],...,0,0,0,0,1,0,0,0,0,2
20482,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,4689616c3d0b46fcba7a771107730791.png,R,CC,3580,2702,BI-RADS 2,DENSITY C,[No Finding],...,0,0,0,0,1,0,0,0,0,2
20483,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,3c22491bcf1d0b004715c28d80981cdd.png,L,CC,3580,2702,BI-RADS 2,DENSITY C,[No Finding],...,0,0,0,0,1,0,0,0,0,2
20484,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,d443b9725e331b8b27589aa725597801.png,R,MLO,3580,2686,BI-RADS 2,DENSITY C,[No Finding],...,0,0,0,0,1,0,0,0,0,2


In [25]:
# Group by 'patient_id', 'image_id', 'laterality', 'view', and 'split', then aggregate 'finding_categories'
aggregated_df = df.groupby(
    ['patient_id', 'image_id', 'laterality', 'view', 'split'])['finding_categories'].agg(
    lambda x: list(set(sum(x, [])))).reset_index()

# Rename the aggregated column for clarity
aggregated_df.rename(columns={'finding_categories': 'text_label'}, inplace=True)


In [26]:
aggregated_df[aggregated_df["patient_id"] == "988ab25980bffad7226b0a8d62a7f499"]

,patient_id,image_id,laterality,view,split,text_label
11736,988ab25980bffad7226b0a8d62a7f499,55e159d8b57f2634f44bd43fd430ff69.png,R,MLO,training,[No Finding]
11737,988ab25980bffad7226b0a8d62a7f499,7c24e6e417f2b9c328f02cf2d96e6684.png,R,CC,training,[No Finding]
11738,988ab25980bffad7226b0a8d62a7f499,ae75f02b14b5582997856eacaf8aa995.png,L,MLO,training,"[Skin Thickening, Mass, Nipple Retraction]"
11739,988ab25980bffad7226b0a8d62a7f499,f2a755158e2f2f4af7d1f8b6257893f5.png,L,CC,training,"[Skin Thickening, Mass, Nipple Retraction]"


In [27]:
# Define a function to filter views and return lists based on the view type
def filter_by_view(group, view_type):
    return group[group['view'] == view_type]['image_id'].tolist(), group[group['view'] == view_type]['text_label'].tolist()

# Group by 'patient_id', 'laterality' and aggregate necessary information including 'split'
new_df = aggregated_df.groupby(['patient_id', 'laterality']).apply(lambda group: pd.Series({
    'split': group['split'].iloc[0],  # Assuming 'split' is consistent within each 'patient_id', 'laterality' group
    'image_ids': group['image_id'].tolist(),
    'views': group['view'].tolist(),
    'cc_image_paths': filter_by_view(group, 'CC')[0],
    'mlo_image_paths': filter_by_view(group, 'MLO')[0],
    'cc_findings': filter_by_view(group, 'CC')[1],
    'mlo_findings': filter_by_view(group, 'MLO')[1]
})).reset_index()

new_df.head(5)

,patient_id,laterality,split,image_ids,views,cc_image_paths,mlo_image_paths,cc_findings,mlo_findings
0,0025a5dc99fd5c742026f0b2b030d3e9,L,test,"[2ddfad7286c2b016931ceccd1e2c7bbc.png, 4515628...","[MLO, CC]",[451562831387e2822923204cf8f0873e.png],[2ddfad7286c2b016931ceccd1e2c7bbc.png],[[No Finding]],[[No Finding]]
1,0025a5dc99fd5c742026f0b2b030d3e9,R,test,"[47c8858666bcce92bcbd57974b5ce522.png, fcf12c2...","[MLO, CC]",[fcf12c2803ba8dc564bf1287c0c97d9a.png],[47c8858666bcce92bcbd57974b5ce522.png],[[No Finding]],[[No Finding]]
2,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,L,training,"[3704f91985dcbc69f6ac2803523d1ecb.png, 7fc1f1b...","[CC, MLO]",[3704f91985dcbc69f6ac2803523d1ecb.png],[7fc1f1bb8bb1a7efaf7104e49c4d8b86.png],[[No Finding]],[[No Finding]]
3,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,R,training,"[16e58fc1d65fa7587247e6224ee96527.png, c4ce686...","[MLO, CC]",[c4ce68631bf70949570ded31a3c69e60.png],[16e58fc1d65fa7587247e6224ee96527.png],[[No Finding]],[[No Finding]]
4,0034765af074f93ed33d5e8399355caf,L,training,"[68f09c18925a66ef2840d4a62f237b31.png, b664cf1...","[CC, MLO]",[68f09c18925a66ef2840d4a62f237b31.png],[b664cf1e7c968896144a3a2005cd3eb4.png],[[No Finding]],[[No Finding]]


In [28]:
new_df[new_df["patient_id"] == "988ab25980bffad7226b0a8d62a7f499"]

,patient_id,laterality,split,image_ids,views,cc_image_paths,mlo_image_paths,cc_findings,mlo_findings
5868,988ab25980bffad7226b0a8d62a7f499,L,training,"[ae75f02b14b5582997856eacaf8aa995.png, f2a7551...","[MLO, CC]",[f2a755158e2f2f4af7d1f8b6257893f5.png],[ae75f02b14b5582997856eacaf8aa995.png],"[[Skin Thickening, Mass, Nipple Retraction]]","[[Skin Thickening, Mass, Nipple Retraction]]"
5869,988ab25980bffad7226b0a8d62a7f499,R,training,"[55e159d8b57f2634f44bd43fd430ff69.png, 7c24e6e...","[MLO, CC]",[7c24e6e417f2b9c328f02cf2d96e6684.png],[55e159d8b57f2634f44bd43fd430ff69.png],[[No Finding]],[[No Finding]]


In [29]:
new_df.to_csv(_dir/'clip_vindr_temp.csv', index=False)

In [30]:
df_new = pd.read_csv(_dir/'clip_vindr_temp.csv')
df_new

,patient_id,laterality,split,image_ids,views,cc_image_paths,mlo_image_paths,cc_findings,mlo_findings
0,0025a5dc99fd5c742026f0b2b030d3e9,L,test,"['2ddfad7286c2b016931ceccd1e2c7bbc.png', '4515...","['MLO', 'CC']",['451562831387e2822923204cf8f0873e.png'],['2ddfad7286c2b016931ceccd1e2c7bbc.png'],[['No Finding']],[['No Finding']]
1,0025a5dc99fd5c742026f0b2b030d3e9,R,test,"['47c8858666bcce92bcbd57974b5ce522.png', 'fcf1...","['MLO', 'CC']",['fcf12c2803ba8dc564bf1287c0c97d9a.png'],['47c8858666bcce92bcbd57974b5ce522.png'],[['No Finding']],[['No Finding']]
2,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,L,training,"['3704f91985dcbc69f6ac2803523d1ecb.png', '7fc1...","['CC', 'MLO']",['3704f91985dcbc69f6ac2803523d1ecb.png'],['7fc1f1bb8bb1a7efaf7104e49c4d8b86.png'],[['No Finding']],[['No Finding']]
3,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,R,training,"['16e58fc1d65fa7587247e6224ee96527.png', 'c4ce...","['MLO', 'CC']",['c4ce68631bf70949570ded31a3c69e60.png'],['16e58fc1d65fa7587247e6224ee96527.png'],[['No Finding']],[['No Finding']]
4,0034765af074f93ed33d5e8399355caf,L,training,"['68f09c18925a66ef2840d4a62f237b31.png', 'b664...","['CC', 'MLO']",['68f09c18925a66ef2840d4a62f237b31.png'],['b664cf1e7c968896144a3a2005cd3eb4.png'],[['No Finding']],[['No Finding']]
...,...,...,...,...,...,...,...,...,...
9995,ff9e4f3aa02c617fd62e2ec6782b6246,R,training,"['d883047653946bff976fcd01f19ca831.png', 'd9f3...","['CC', 'MLO']",['d883047653946bff976fcd01f19ca831.png'],['d9f3cf3bb995aa83a016da75ec8ef3f0.png'],[['No Finding']],[['No Finding']]
9996,ffe7a45f8390f242db3b843762a4a7aa,L,training,"['3d9c7d682f61c06ecc14a9c17a14ab32.png', 'f726...","['MLO', 'CC']",['f726bfd1a19108301f5bd41e86e2833a.png'],['3d9c7d682f61c06ecc14a9c17a14ab32.png'],[['No Finding']],[['No Finding']]
9997,ffe7a45f8390f242db3b843762a4a7aa,R,training,"['a96fa10911477260b7758a2dbe69bc99.png', 'd2f6...","['MLO', 'CC']",['d2f6630d1b13d6af9d2563b11293f0f3.png'],['a96fa10911477260b7758a2dbe69bc99.png'],[['No Finding']],[['No Finding']]
9998,fff2339ea4b5d2f1792672ba7d52b318,L,training,"['5144bf29398269fa2cf8c36b9c6db7f3.png', 'e419...","['MLO', 'CC']",['e4199214f5b40bd40847f5c2aedc44ef.png'],['5144bf29398269fa2cf8c36b9c6db7f3.png'],[['No Finding']],[['No Finding']]


In [31]:
# Define the total list of possible findings
total_findings = ['Mass', 'Suspicious Calcification', 'Architectural Distortion', 'Global Asymmetry', 
                  'Focal Asymmetry', 'Asymmetry', 'Suspicious Lymph Node', 'Nipple Retraction', 
                  'Skin Retraction', 'Skin Thickening',]

# Function to create the specified format for cc_findings
def create_cc_findings(row):
    # Initialize all slots with empty lists
    findings = [[], [], [], []]
    
    # Determine the index for positive and negative findings based on laterality
    positive_index = 0 if row['laterality'] == 'R' else 1
    negative_index = 2 if row['laterality'] == 'R' else 3
    
    # Place the positive findings in the correct slot
    findings[positive_index] = row['cc_findings']
    
    # Calculate the negative findings
    negative_findings = list(set(total_findings) - set(row['cc_findings']))
    
    # Place the negative findings in the correct slot
    findings[negative_index] = negative_findings
    
    return findings

def create_mlo_findings(row):
    # Initialize all slots with empty lists
    findings = [[], [], [], []]
    
    # Determine the index for positive and negative findings based on laterality
    positive_index = 0 if row['laterality'] == 'R' else 1
    negative_index = 2 if row['laterality'] == 'R' else 3
    
    # Place the positive findings in the correct slot
    findings[positive_index] = row['mlo_findings']
    
    # Calculate the negative findings
    negative_findings = list(set(total_findings) - set(row['mlo_findings']))
    
    # Place the negative findings in the correct slot
    findings[negative_index] = negative_findings
    
    return findings

# Apply the function to each row in the dataframe
df_new['cc_findings_formatted'] = df_new.apply(create_cc_findings, axis=1)
df_new['mlo_findings_formatted'] = df_new.apply(create_mlo_findings, axis=1)

df_new

,patient_id,laterality,split,image_ids,views,cc_image_paths,mlo_image_paths,cc_findings,mlo_findings,cc_findings_formatted,mlo_findings_formatted
0,0025a5dc99fd5c742026f0b2b030d3e9,L,test,"['2ddfad7286c2b016931ceccd1e2c7bbc.png', '4515...","['MLO', 'CC']",['451562831387e2822923204cf8f0873e.png'],['2ddfad7286c2b016931ceccd1e2c7bbc.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A..."
1,0025a5dc99fd5c742026f0b2b030d3e9,R,test,"['47c8858666bcce92bcbd57974b5ce522.png', 'fcf1...","['MLO', 'CC']",['fcf12c2803ba8dc564bf1287c0c97d9a.png'],['47c8858666bcce92bcbd57974b5ce522.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm..."
2,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,L,training,"['3704f91985dcbc69f6ac2803523d1ecb.png', '7fc1...","['CC', 'MLO']",['3704f91985dcbc69f6ac2803523d1ecb.png'],['7fc1f1bb8bb1a7efaf7104e49c4d8b86.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A..."
3,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,R,training,"['16e58fc1d65fa7587247e6224ee96527.png', 'c4ce...","['MLO', 'CC']",['c4ce68631bf70949570ded31a3c69e60.png'],['16e58fc1d65fa7587247e6224ee96527.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm..."
4,0034765af074f93ed33d5e8399355caf,L,training,"['68f09c18925a66ef2840d4a62f237b31.png', 'b664...","['CC', 'MLO']",['68f09c18925a66ef2840d4a62f237b31.png'],['b664cf1e7c968896144a3a2005cd3eb4.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,ff9e4f3aa02c617fd62e2ec6782b6246,R,training,"['d883047653946bff976fcd01f19ca831.png', 'd9f3...","['CC', 'MLO']",['d883047653946bff976fcd01f19ca831.png'],['d9f3cf3bb995aa83a016da75ec8ef3f0.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm..."
9996,ffe7a45f8390f242db3b843762a4a7aa,L,training,"['3d9c7d682f61c06ecc14a9c17a14ab32.png', 'f726...","['MLO', 'CC']",['f726bfd1a19108301f5bd41e86e2833a.png'],['3d9c7d682f61c06ecc14a9c17a14ab32.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A..."
9997,ffe7a45f8390f242db3b843762a4a7aa,R,training,"['a96fa10911477260b7758a2dbe69bc99.png', 'd2f6...","['MLO', 'CC']",['d2f6630d1b13d6af9d2563b11293f0f3.png'],['a96fa10911477260b7758a2dbe69bc99.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm..."
9998,fff2339ea4b5d2f1792672ba7d52b318,L,training,"['5144bf29398269fa2cf8c36b9c6db7f3.png', 'e419...","['MLO', 'CC']",['e4199214f5b40bd40847f5c2aedc44ef.png'],['5144bf29398269fa2cf8c36b9c6db7f3.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A..."


In [32]:
# Add a 'fold' column based on the value of 'split'
df_new['fold'] = df_new['split'].apply(lambda x: 0 if x == 'test' else 1)


df_new

,patient_id,laterality,split,image_ids,views,cc_image_paths,mlo_image_paths,cc_findings,mlo_findings,cc_findings_formatted,mlo_findings_formatted,fold
0,0025a5dc99fd5c742026f0b2b030d3e9,L,test,"['2ddfad7286c2b016931ceccd1e2c7bbc.png', '4515...","['MLO', 'CC']",['451562831387e2822923204cf8f0873e.png'],['2ddfad7286c2b016931ceccd1e2c7bbc.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",0
1,0025a5dc99fd5c742026f0b2b030d3e9,R,test,"['47c8858666bcce92bcbd57974b5ce522.png', 'fcf1...","['MLO', 'CC']",['fcf12c2803ba8dc564bf1287c0c97d9a.png'],['47c8858666bcce92bcbd57974b5ce522.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",0
2,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,L,training,"['3704f91985dcbc69f6ac2803523d1ecb.png', '7fc1...","['CC', 'MLO']",['3704f91985dcbc69f6ac2803523d1ecb.png'],['7fc1f1bb8bb1a7efaf7104e49c4d8b86.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1
3,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,R,training,"['16e58fc1d65fa7587247e6224ee96527.png', 'c4ce...","['MLO', 'CC']",['c4ce68631bf70949570ded31a3c69e60.png'],['16e58fc1d65fa7587247e6224ee96527.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1
4,0034765af074f93ed33d5e8399355caf,L,training,"['68f09c18925a66ef2840d4a62f237b31.png', 'b664...","['CC', 'MLO']",['68f09c18925a66ef2840d4a62f237b31.png'],['b664cf1e7c968896144a3a2005cd3eb4.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ff9e4f3aa02c617fd62e2ec6782b6246,R,training,"['d883047653946bff976fcd01f19ca831.png', 'd9f3...","['CC', 'MLO']",['d883047653946bff976fcd01f19ca831.png'],['d9f3cf3bb995aa83a016da75ec8ef3f0.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1
9996,ffe7a45f8390f242db3b843762a4a7aa,L,training,"['3d9c7d682f61c06ecc14a9c17a14ab32.png', 'f726...","['MLO', 'CC']",['f726bfd1a19108301f5bd41e86e2833a.png'],['3d9c7d682f61c06ecc14a9c17a14ab32.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1
9997,ffe7a45f8390f242db3b843762a4a7aa,R,training,"['a96fa10911477260b7758a2dbe69bc99.png', 'd2f6...","['MLO', 'CC']",['d2f6630d1b13d6af9d2563b11293f0f3.png'],['a96fa10911477260b7758a2dbe69bc99.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1
9998,fff2339ea4b5d2f1792672ba7d52b318,L,training,"['5144bf29398269fa2cf8c36b9c6db7f3.png', 'e419...","['MLO', 'CC']",['e4199214f5b40bd40847f5c2aedc44ef.png'],['5144bf29398269fa2cf8c36b9c6db7f3.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1


In [33]:
# Rename the specified columns
df_new.rename(columns={
    'image_ids': 'image', 
    'cc_image_paths': 'CC',
    'mlo_image_paths': 'MLO',
    'cc_findings_formatted': 'CC_FINDING',
    'mlo_findings_formatted': 'MLO_FINDING'
}, inplace=True)

# Display the first few rows to verify the column name changes
df_new

,patient_id,laterality,split,image,views,CC,MLO,cc_findings,mlo_findings,CC_FINDING,MLO_FINDING,fold
0,0025a5dc99fd5c742026f0b2b030d3e9,L,test,"['2ddfad7286c2b016931ceccd1e2c7bbc.png', '4515...","['MLO', 'CC']",['451562831387e2822923204cf8f0873e.png'],['2ddfad7286c2b016931ceccd1e2c7bbc.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",0
1,0025a5dc99fd5c742026f0b2b030d3e9,R,test,"['47c8858666bcce92bcbd57974b5ce522.png', 'fcf1...","['MLO', 'CC']",['fcf12c2803ba8dc564bf1287c0c97d9a.png'],['47c8858666bcce92bcbd57974b5ce522.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",0
2,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,L,training,"['3704f91985dcbc69f6ac2803523d1ecb.png', '7fc1...","['CC', 'MLO']",['3704f91985dcbc69f6ac2803523d1ecb.png'],['7fc1f1bb8bb1a7efaf7104e49c4d8b86.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1
3,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,R,training,"['16e58fc1d65fa7587247e6224ee96527.png', 'c4ce...","['MLO', 'CC']",['c4ce68631bf70949570ded31a3c69e60.png'],['16e58fc1d65fa7587247e6224ee96527.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1
4,0034765af074f93ed33d5e8399355caf,L,training,"['68f09c18925a66ef2840d4a62f237b31.png', 'b664...","['CC', 'MLO']",['68f09c18925a66ef2840d4a62f237b31.png'],['b664cf1e7c968896144a3a2005cd3eb4.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ff9e4f3aa02c617fd62e2ec6782b6246,R,training,"['d883047653946bff976fcd01f19ca831.png', 'd9f3...","['CC', 'MLO']",['d883047653946bff976fcd01f19ca831.png'],['d9f3cf3bb995aa83a016da75ec8ef3f0.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1
9996,ffe7a45f8390f242db3b843762a4a7aa,L,training,"['3d9c7d682f61c06ecc14a9c17a14ab32.png', 'f726...","['MLO', 'CC']",['f726bfd1a19108301f5bd41e86e2833a.png'],['3d9c7d682f61c06ecc14a9c17a14ab32.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1
9997,ffe7a45f8390f242db3b843762a4a7aa,R,training,"['a96fa10911477260b7758a2dbe69bc99.png', 'd2f6...","['MLO', 'CC']",['d2f6630d1b13d6af9d2563b11293f0f3.png'],['a96fa10911477260b7758a2dbe69bc99.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1
9998,fff2339ea4b5d2f1792672ba7d52b318,L,training,"['5144bf29398269fa2cf8c36b9c6db7f3.png', 'e419...","['MLO', 'CC']",['e4199214f5b40bd40847f5c2aedc44ef.png'],['5144bf29398269fa2cf8c36b9c6db7f3.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1


In [64]:
df_new['patient_id_laterality'] = df_new['patient_id'].astype(str) + '_' + df_new['laterality']
df_new

,patient_id,laterality,split,image,views,CC,MLO,cc_findings,mlo_findings,CC_FINDING,MLO_FINDING,fold,patient_id_laterality
0,0025a5dc99fd5c742026f0b2b030d3e9,L,test,"['2ddfad7286c2b016931ceccd1e2c7bbc.png', '4515...","['MLO', 'CC']",['451562831387e2822923204cf8f0873e.png'],['2ddfad7286c2b016931ceccd1e2c7bbc.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",0,0025a5dc99fd5c742026f0b2b030d3e9_L
1,0025a5dc99fd5c742026f0b2b030d3e9,R,test,"['47c8858666bcce92bcbd57974b5ce522.png', 'fcf1...","['MLO', 'CC']",['fcf12c2803ba8dc564bf1287c0c97d9a.png'],['47c8858666bcce92bcbd57974b5ce522.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",0,0025a5dc99fd5c742026f0b2b030d3e9_R
2,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,L,training,"['3704f91985dcbc69f6ac2803523d1ecb.png', '7fc1...","['CC', 'MLO']",['3704f91985dcbc69f6ac2803523d1ecb.png'],['7fc1f1bb8bb1a7efaf7104e49c4d8b86.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1,0028fb2c7f0b3a5cb9a80cb0e1cdbb91_L
3,0028fb2c7f0b3a5cb9a80cb0e1cdbb91,R,training,"['16e58fc1d65fa7587247e6224ee96527.png', 'c4ce...","['MLO', 'CC']",['c4ce68631bf70949570ded31a3c69e60.png'],['16e58fc1d65fa7587247e6224ee96527.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1,0028fb2c7f0b3a5cb9a80cb0e1cdbb91_R
4,0034765af074f93ed33d5e8399355caf,L,training,"['68f09c18925a66ef2840d4a62f237b31.png', 'b664...","['CC', 'MLO']",['68f09c18925a66ef2840d4a62f237b31.png'],['b664cf1e7c968896144a3a2005cd3eb4.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1,0034765af074f93ed33d5e8399355caf_L
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ff9e4f3aa02c617fd62e2ec6782b6246,R,training,"['d883047653946bff976fcd01f19ca831.png', 'd9f3...","['CC', 'MLO']",['d883047653946bff976fcd01f19ca831.png'],['d9f3cf3bb995aa83a016da75ec8ef3f0.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1,ff9e4f3aa02c617fd62e2ec6782b6246_R
9996,ffe7a45f8390f242db3b843762a4a7aa,L,training,"['3d9c7d682f61c06ecc14a9c17a14ab32.png', 'f726...","['MLO', 'CC']",['f726bfd1a19108301f5bd41e86e2833a.png'],['3d9c7d682f61c06ecc14a9c17a14ab32.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1,ffe7a45f8390f242db3b843762a4a7aa_L
9997,ffe7a45f8390f242db3b843762a4a7aa,R,training,"['a96fa10911477260b7758a2dbe69bc99.png', 'd2f6...","['MLO', 'CC']",['d2f6630d1b13d6af9d2563b11293f0f3.png'],['a96fa10911477260b7758a2dbe69bc99.png'],[['No Finding']],[['No Finding']],"[[['No Finding']], [], [Skin Thickening, Asymm...","[[['No Finding']], [], [Skin Thickening, Asymm...",1,ffe7a45f8390f242db3b843762a4a7aa_R
9998,fff2339ea4b5d2f1792672ba7d52b318,L,training,"['5144bf29398269fa2cf8c36b9c6db7f3.png', 'e419...","['MLO', 'CC']",['e4199214f5b40bd40847f5c2aedc44ef.png'],['5144bf29398269fa2cf8c36b9c6db7f3.png'],[['No Finding']],[['No Finding']],"[[], [['No Finding']], [], [Skin Thickening, A...","[[], [['No Finding']], [], [Skin Thickening, A...",1,fff2339ea4b5d2f1792672ba7d52b318_L


In [68]:
df_new.drop(index=8550, inplace=True)

In [69]:
df_final = df_new

In [71]:
df_final.rename(columns={
    'views': 'view', 
}, inplace=True)

In [75]:
# Convert the string representations of the lists into actual lists
df_final['view'] = df_final['view'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_final['image'] = df_final['image'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Define a function to reorder based on 'CC' and 'MLO' sequence and adjust other columns accordingly
def reorder_sequence(row):
    # Create a mapping of view to index
    view_index_map = {'CC': 0, 'MLO': 1}
    # Initialize the new order for views and related columns
    new_order = ['CC', 'MLO']
    new_image = [None, None]

    # Go through the existing views and place the items in the correct order
    for i, view in enumerate(row['view']):
        index = view_index_map.get(view)
        if index is not None:
            new_image[index] = row['image'][i]

    # Assign the reordered lists back to the row
    row['view'] = new_order
    row['image'] = new_image

    return row

# Apply the reordering function to each row
df_final = df_final.apply(reorder_sequence, axis=1)


In [74]:
df_final.to_csv(_dir/'clip_vindr_final.csv', index=False)